# Tutorial: Embedding StableHLO in Jax
This tutorial has a hard dependency of python. Python 3.11 is required to have enzyme-ad of version 0.0.9, overwise we get version 0.0.6 which is broken

In [1]:
!python -V

Python 3.11.11


In [8]:
!pip install juliacall
!pip install enzyme-ad
!pip install jax


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Loading the code
To generate the hlo model, you have to run the export_model.jl notebook.

In [27]:
with open("exported_lux_model.mlir") as file:
    code = file.read()
code

'module @reactant_apply attributes {mhlo.num_partitions = 1 : i64, mhlo.num_replicas = 1 : i64} {\n  func.func @main(%arg0: tensor<10x55x6xf32>, %arg1: tensor<6x7xf32>, %arg2: tensor<7xf32>, %arg3: tensor<7x4xf32>, %arg4: tensor<4xf32>, %arg5: tensor<1x4xf32>, %arg6: tensor<1x4xf32>, %arg7: tensor<4x6xf32>, %arg8: tensor<6xf32>, %arg9: tensor<1x6xf32>, %arg10: tensor<1x6xf32>, %arg11: tensor<6x1xf32>, %arg12: tensor<1xf32> {reactant.donated}) -> tensor<10x1xf32> {\n    %c = stablehlo.constant dense<"..."> : tensor<550x3xi64>\n    %cst = stablehlo.constant dense<0.000000e+00> : tensor<6x10xf32>\n    %cst_0 = stablehlo.constant dense<6.000000e+00> : tensor<4x10xf32>\n    %cst_1 = stablehlo.constant dense<0.000000e+00> : tensor<4x10xf32>\n    %cst_2 = stablehlo.constant dense<0.000000e+00> : tensor<4x550xf32>\n    %cst_3 = stablehlo.constant dense<0.000000e+00> : tensor<7x550xf32>\n    %cst_4 = stablehlo.constant dense<6.000000e+01> : tensor<f32>\n    %cst_5 = stablehlo.constant dense<9.9

# Loading weights
Weights are serilalized in a julia SerializedModel object. Since the serialized object reference some code from MLNanoShaperRunner, we have to lode MLNanoShaperRunner beforehand.

In [ ]:
from juliacall import Main as jl
from pathlib import Path
jl.seval("using Pkg")
jl.Pkg.develop(path ="../MLNanoShaperRunner")
jl.seval("using Serialization,MLNanoShaperRunner")
home = Path.home()
path = home /'datasets'/'models'/'tiny_angular_dense_s_jobs_14_6_3_c_2025-03-19_epoch_400_9592899277305186470'

   Resolving package versions...
    Updating `~/workspace/MLNanoShaper.jl/notebooks/.venv/julia_env/Project.toml`
  [0d104b66] + MLNanoShaperRunner v0.1.0 `../../../MLNanoShaperRunner`
    Updating `~/workspace/MLNanoShaper.jl/notebooks/.venv/julia_env/Manifest.toml`
  [47edcb42] + ADTypes v1.14.0
  [7d9f7c33] + Accessors v0.1.42
  [79e6a3ab] + Adapt v4.3.0
  [dce04be8] + ArgCheck v2.4.0
  [4fba245c] + ArrayInterface v7.18.0
  [a9b6321e] + Atomix v1.1.1
  [198e06fe] + BangBang v0.4.4
  [9718e550] + Baselet v0.1.1
  [47718e42] + BioGenerics v0.1.5
  [de9282ab] + BioStructures v4.4.1
  [3c28c6f8] + BioSymbols v5.1.3
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.6
⌅ [8ce10254] + Bumper v0.6.0
  [2a0fbf3d] + CPUSummary v0.2.6
  [d360d2e6] + ChainRulesCore v1.25.1
  [fb6a15b2] + CloseOpenIntervals v0.1.13
  [944b1d66] + CodecZlib v0.7.8
  [bbf7d656] + CommonSubexpressions v0.3.1
  [f70d9fcc] + CommonWorldInvalidations v1.0.0
  [34da2185] + Compat v4.16.0
  [a33af91c] + CompositionsB

In [25]:
import jax.numpy as jnp
def convert_to_tensors(jl_array):
    res = jnp.array(jl_array)
    rank = res.ndim
    return jnp.transpose(res)
def extact_dense_layer(parameters):
    weight = convert_to_tensors(parameters.weight)
    bias = convert_to_tensors(parameters.bias)
    return [weight,bias]
def extact_layer_norm(parameters):
    bias = convert_to_tensors(parameters.bias)
    scale = convert_to_tensors(parameters.scale)
    return [bias,scale]

def extract_parameters(parameters):
    par1w,par1b = extact_dense_layer(parameters.layer_2.layer_2.layer_1.layer_1)
    par2w,par2b = extact_dense_layer(parameters.layer_2.layer_2.layer_1.layer_2)
    par3b,par3s = extact_layer_norm(parameters.layer_3.layer_2)
    par4w,par4b = extact_dense_layer(parameters.layer_3.layer_3)
    par5b,par5s = extact_layer_norm(parameters.layer_3.layer_4)
    par6w,par6b = extact_dense_layer(parameters.layer_3.layer_5)
    return [
        par1w,
        par1b,
        par2w,
        par2b,
        par3b,
        par3s,
        par4w,
        par4b,
        par5b,
        par5s,
        par6w,
        par6b,
    ]
params = extract_parameters(jl.deserialize(str(path)).parameters)
dummy_imputs = jax.random.normal(jax.random.PRNGKey(0), (10,55,6))
params


[Array([[ 0.02676329, -0.68465745, -0.43417034,  0.36572096, -0.06240562,
         -0.04804124, -0.6535572 ],
        [ 0.00969661,  0.76495713,  0.5380483 ,  0.32660553,  0.13679862,
         -1.0041145 ,  0.04248108],
        [-0.03517471,  0.01975672,  0.37579754,  0.59486717,  0.13285469,
          0.5916052 ,  0.06602191],
        [-0.0022126 , -0.15309426, -0.28085855,  1.6166728 , -0.20440318,
          1.2361188 ,  0.06423712],
        [-0.02103094,  0.36936328,  0.4337348 ,  0.06132033,  0.06723495,
         -0.6151614 , -0.11044361],
        [-0.0036585 , -0.8173947 ,  0.69474036, -0.8468983 ,  0.33898   ,
          0.81042975,  0.29406574]], dtype=float32),
 Array([ 0.00713245, -0.08776207, -0.0788786 ,  0.42167863, -0.00696724,
        -0.30991384,  0.05974844], dtype=float32),
 Array([[ 0.00807936,  0.051105  ,  0.00539792, -0.00324079],
        [-0.3936812 , -0.11908815, -0.50028217,  0.11244424],
        [ 0.46386185,  0.11883527,  0.07655747,  0.28824443],
        [ 0.2

# Compiling and running
Once we have the model, and the parameters we can run the model on the jax backend

In [29]:
from enzyme_ad.jax import hlo_call
import jax
@jax.jit
def run_lux_model(
    x,
    *args
):
    return hlo_call(
        x,
        *args,
        source=code,
    )
run_lux_model(dummy_imputs,*params)

[Array([[0.5182601 ],
        [0.9988796 ],
        [0.5825275 ],
        [0.99655426],
        [0.0654102 ],
        [0.95935065],
        [0.9942368 ],
        [0.99813014],
        [0.99938834],
        [0.99396306]], dtype=float32)]